Practice homework:

Spam dataset: 
https://www.kaggle.com/uciml/sms-spam-collection-dataset

### Features: 
* Number of words
* "bag of words" --
** Split text into words, remove punctuation, lower case ✓
** Create a matrix where every word is a column ✓
** Values: word count in a message or 0 ✓

** Think about feature selection -- using coorrelation or P-value or chi-squarred or??? ✓

** Train and test an SVM; do a few iterations to refine feature selection ✓

#0. Import the data

In [ ]:
# Mount data drive
from google.colab import drive
drive.mount('/data/')
data_dir = '/data/My Drive/EMSE 6575/SVM Homework'

Drive already mounted at /data/; to attempt to forcibly remount, call drive.mount("/data/", force_remount=True).


In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df = pd.read_excel(data_dir + "/spam_resaved.xlsx")
df = df[['v1', 'v2']]
df.columns = ['category', 'text']
df['doc_id'] = df.index
df['doc_id'] = df['doc_id'].apply(lambda x: "doc_" + str(x))
print(df['category'].value_counts())
df.head()

ham     4825
spam     747
Name: category, dtype: int64


,category,text,doc_id
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",doc_0
1,ham,Ok lar... Joking wif u oni...,doc_1
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,doc_2
3,ham,U dun say so early hor... U c already then say...,doc_3
4,ham,"Nah I don't think he goes to usf, he lives around here though",doc_4


#1. Tokenize and clean the data

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Minimal example
print(df['text'][0])
tokens = str(df['text'][0]).split()
tokens = [token.lower() for token in tokens]
tokens = [re.sub(r'[^\w\s]','',token) for token in tokens]
tokens = [token for token in tokens if len(token) >= 3]
tokens = [token for token in tokens if not token in stopwords.words()]
tokens

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


['jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'buffet',
 'got',
 'amore']

In [ ]:
token_dfs = []
for i in range(0, len(df)):
  # split the text into tokens
  tokens = str(df['text'][i]).split()
  # drop case
  tokens = [token.lower() for token in tokens]
  
  # create a tidy structure for the parsed and cleaned tokens
  temp = pd.DataFrame({'token': tokens})
  
  # append  unique document id and text category
  temp['doc_id'] = df['doc_id'][i]
  temp['category'] = df['category'][i]

  # collect the temp token dataframes
  token_dfs.append(temp)

token_df = pd.concat(token_dfs)
print("Initial token count: " + str(len(token_df)))

#remove stop words
token_df = token_df[token_df['token'].isin(stopwords.words()) == False]
print("Drop stop words: " + str(len(token_df)))

# drop puncuation
token_df['token'] = token_df['token'].apply(lambda x: re.sub(r'[^\w\s]','',x))  

#remove rare words
rare_words = token_df['token'].value_counts().reset_index()
rare_words.columns = ['term', 'count']         
rare_words = rare_words['term'][rare_words['count'] <= 4].tolist()
token_df = token_df[token_df['token'].isin(rare_words) == False]
print("Drop rare words: " + str(len(token_df)))

#remove short words
token_df = token_df[token_df['token'].apply(len) >= 3]
print("Drop short words: " + str(len(token_df)))

token_df.head()

Initial token count: 86335
Drop stop words: 51730
Drop rare words: 40001
Drop short words: 33900


,token,doc_id,category
3,point,doc_0,ham
4,crazy,doc_0,ham
5,available,doc_0,ham
8,bugis,doc_0,ham
10,great,doc_0,ham


# 2. Create a term frequency table

In [ ]:
token_df['token'][df['category'] == 'spam'].value_counts().head(20)

call      198
get       128
know      107
like       82
got        81
ltgt       79
send       76
free       76
now        73
going      64
text       64
you        62
time       62
need       61
mobile     60
good       55
lor        54
home       53
day        52
back       52
Name: token, dtype: int64

In [ ]:
token_df['token'][df['category'] == 'ham'].value_counts().head(20)

call     378
get      258
free     199
ltgt     197
good     179
like     160
now      158
got      157
day      150
know     150
love     147
time     146
ill      146
you      145
sorry    130
text     124
stop     118
txt      117
send     114
home     109
Name: token, dtype: int64

In [ ]:
# drop common words seen in each category
common_words = ['call', 'get', 'free', 'ltgt', 'text', 'you', 'got', 'like',
                'send', 'now', 'txt', 'dont']
token_df = token_df[token_df['token'].isin(common_words) == False]
print("Drop common words across data categories: " + str(len(token_df)))

Drop common words across data categories: 30783


In [ ]:
token_df['token'][df['category'] == 'spam'].value_counts().head(20)

know      107
going      64
time       62
need       61
mobile     60
good       55
lor        54
home       53
back       52
day        52
ill        51
see        50
still      49
love       48
today      47
week       45
right      45
make       43
way        43
phone      42
Name: token, dtype: int64

In [ ]:
term_freq = (token_df
              .groupby(['doc_id', 'token'], as_index = False)
              .count())
term_freq.columns = ['doc_id', 'term', 'term_count']
term_freq.sort_values(by=['term_count'], ascending=False)

,doc_id,term,term_count
11892,doc_3015,happy,15
18059,doc_409,missing,6
854,doc_1139,missing,6
11888,doc_3015,day,6
3724,doc_1621,simple,5
...,...,...,...
10034,doc_2700,frnds,1
10033,doc_2700,feb,1
10031,doc_2700,day,1
10030,doc_2700,comes,1


In [ ]:
df[df['doc_id'] == 'doc_1139']

,category,text,doc_id
1139,ham,Message:some text missing* Sender:Name Missing* *Number Missing *Sent:Date missing *Missing U a lot thats y everything is missing sent via fullonsms.com,doc_1139


# 3. Reshape the term frequency table to a document term matrix

In [ ]:
doc_list = term_freq['doc_id'].unique().tolist()
doc_list = doc_list[0:3] # minimal example to verify code is working

reshape_dfs =[]
for doc in doc_list:
  temp = term_freq[['term_count']][term_freq['doc_id'] == doc].T
  temp.columns = term_freq['term'][term_freq['doc_id'] == doc].tolist()
  temp.index = [doc]
  reshape_dfs.append(temp)

doc_term_mat = pd.concat(reshape_dfs)
doc_term_mat = doc_term_mat.fillna(0)
doc_term_mat

,available,bugis,crazy,great,point,wat,world,joking,lar,wif,anymore,enough,gonna,home,ive,soon,stuff,talk,today,tonight
doc_0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
doc_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
doc_10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
import time
from datetime import timedelta

doc_list = term_freq['doc_id'].unique().tolist()
reshape_dfs =[]
counter = 0
for doc in doc_list:
  counter += 1
  if counter % 500 == 0:
    print(str(counter) + " complete out of " + str(len(doc_list)) + " documents")
  temp = term_freq[['term_count']][term_freq['doc_id'] == doc].T
  temp.columns = term_freq['term'][term_freq['doc_id'] == doc].tolist()
  temp.index = [doc]
  reshape_dfs.append(temp)

print("\n merging stuff")
start_time = time.time()
doc_term_mat = pd.concat(reshape_dfs)
doc_term_mat = doc_term_mat.fillna(0)

print("--- %s time elapsed ---" % str(timedelta(seconds=time.time() - start_time)))
print(doc_term_mat.shape)

500 complete out of 5386 documents
1000 complete out of 5386 documents
1500 complete out of 5386 documents
2000 complete out of 5386 documents
2500 complete out of 5386 documents
3000 complete out of 5386 documents
3500 complete out of 5386 documents
4000 complete out of 5386 documents
4500 complete out of 5386 documents
5000 complete out of 5386 documents

 merging stuff
--- 0:06:29.792204 time elapsed ---
(5386, 1591)


# 4. Determine what terms matter for predicting spam

Remove highly correlated features

In [ ]:
doc_term_mat2 = doc_term_mat
doc_term_mat2['doc_id'] = doc_term_mat2.index
doc_term_mat2 = doc_term_mat2.merge(df[['doc_id', 'category']], how = 'left', on = 'doc_id')

In [ ]:
def dichotomize_cat(txt):
  x = 0
  if txt == "spam":
    x = 1
  return x

doc_term_mat2['category'] = doc_term_mat2['category'].apply(lambda x: dichotomize_cat(x))
doc_term_mat2['category'].value_counts()

0    4643
1     743
Name: category, dtype: int64

In [ ]:
cols = list(doc_term_mat2)
cols.insert(0, cols.pop(cols.index('doc_id')))
cols.insert(1, cols.pop(cols.index('category')))
doc_term_mat2 = doc_term_mat2[cols]

keep only significant variables

In [ ]:
Y = doc_term_mat2.loc[:, 'category']  # all rows of 'diagnosis' 
X = doc_term_mat2.drop(['doc_id', 'category'], axis=1)

from sklearn.model_selection import train_test_split as tts
X.insert(loc=len(X.columns), column='intercept', value=1) #### column of 1's
X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)

In [ ]:
import statsmodels.api as sm
regression_ols = sm.OLS(y_train, X_train).fit()
pvalues = regression_ols.pvalues 
sig_vars = []
for i in range(0, len(pvalues)):
  if pvalues[i] <= 0.5:
    sig_vars.append(pvalues.index[i])
print(len(sig_vars))
sig_vars[0:20]

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


850


['crazy',
 'wat',
 'lar',
 'enough',
 'gonna',
 'home',
 'ive',
 'talk',
 'today',
 'long',
 'pick',
 'price',
 'heard',
 'tat',
 'cash',
 'guaranteed',
 'please',
 'prize',
 'representative',
 'service']

In [ ]:
X_train = X_train[sig_vars]
X_test = X_test[sig_vars]

regression_ols = sm.OLS(y_train, X_train).fit()
pvalues = regression_ols.pvalues 
sig_vars = []
for i in range(0, len(pvalues)):
  if pvalues[i] <= 0.1:
    sig_vars.append(pvalues.index[i])
print(len(sig_vars))
sig_vars[0:20]

480


['crazy',
 'enough',
 'gonna',
 'today',
 'pick',
 'price',
 'heard',
 'cash',
 'guaranteed',
 'please',
 'prize',
 'representative',
 'service',
 'â5000',
 'told',
 'give',
 '08000839402',
 'mobileupd8',
 'nokia',
 'orange']

In [ ]:
reg_strength = 1000

### Formula in Slide 8
def compute_cost(W, X, Y):
  N = X.shape[0]
  distances = 1 - Y * (np.dot(X, W))
  distances[distances < 0] = 0
  hinge_loss = reg_strength * (np.sum(distances) / N)
  cost = 1 / 2 * np.dot(W, W) + hinge_loss
  return(cost)

def calculate_cost_gradient(W, X_batch, Y_batch):
    if type(Y_batch) != np.array: #float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])

    distance = 1 - (Y_batch * np.dot(X_batch, W))
#    if type(distance)== np.float64:
#      distance = np.array([distance])
    dw = np.zeros(len(W))

    #### for every dimension, apply formula in Slide 9
    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (reg_strength * Y_batch[ind] * X_batch[ind])
        dw += di  
    dw = dw/len(Y_batch)  # average
    return dw

from sklearn.utils import shuffle
def sgd(features, outputs, max_epochs=5000, learning_rate=0.05,cost_threshold = 0.01):
  weights = np.zeros(features.shape[1])
  prev_cost = float("inf")
  nth = 0
  for epoch in range(1, max_epochs):
    X, Y = shuffle(features, outputs)
    for ind, x in enumerate(X):  #### itereate through the dimensions
      ascent = calculate_cost_gradient(weights, x, Y[ind])
      weights = weights - (learning_rate * ascent)

    if epoch == 2 ** nth or epoch == max_epochs - 1:
      cost = compute_cost(weights, features, outputs)
      print("Epoch is:{} and Cost is: {}".format(epoch, cost))

      #### If diffrence in the cost from last ^2 iterations to now is < 1%: get out
      if abs(prev_cost - cost) < cost_threshold * prev_cost:
            return weights
      prev_cost = cost
      nth += 1
  return weights

In [ ]:
import numpy as np
X_train = X_train[sig_vars]
X_test = X_test[sig_vars]

W = sgd(X_train.to_numpy(), y_train.to_numpy())

from sklearn.metrics import accuracy_score, recall_score, precision_score 
def test():
  y_test_predicted = np.array([])
  for i in range(X_test.shape[0]):
      yp = np.sign(np.dot(W, X_test.to_numpy()[i])) #model
      y_test_predicted = np.append(y_test_predicted, yp)
  print("accuracy on test dataset: {}".format(accuracy_score(y_test.to_numpy(), y_test_predicted)))
  print("recall on test dataset: {}".format(recall_score(y_test.to_numpy(), y_test_predicted)))
  print("precision on test dataset: {}".format(precision_score(y_test.to_numpy(), y_test_predicted)))

test()

Epoch is:1 and Cost is: 910.1495237304057
Epoch is:2 and Cost is: 872.1369238099027
Epoch is:4 and Cost is: 961.898403646313
Epoch is:8 and Cost is: 872.3843487732357
Epoch is:16 and Cost is: 1008.4441357908554
Epoch is:32 and Cost is: 2026.9460913385926
Epoch is:64 and Cost is: 931.1829656458793
Epoch is:128 and Cost is: 4352.157332200734
Epoch is:256 and Cost is: 5344.417652885657
Epoch is:512 and Cost is: 1145.647817346647
Epoch is:1024 and Cost is: 866.1646721692231
Epoch is:2048 and Cost is: 994.1051022919514
Epoch is:4096 and Cost is: 1240.126225004275
Epoch is:4999 and Cost is: 2841.85000390915
accuracy on test dataset: 0.14285714285714285
recall on test dataset: 1.0
precision on test dataset: 0.14285714285714285


^^ This is a very terrible model.  14% precision means we incorrectly predict spam all the time. Now we will try the sklearn version.

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from statistics import mean
from numpy import std

def eval_model(model, print_text, X, y):
  cv = KFold(n_splits=10) 
  accuracy = cross_val_score(model, X, y, scoring = 'accuracy', cv = cv)
  accuracy = list(accuracy)
  f1_scores = cross_val_score(model, X, y, scoring = 'f1', cv = cv)
  f1_scores = list(f1_scores)
  precision_scores = cross_val_score(model, X, y, scoring = 'precision', cv = cv)
  precision_scores = list(precision_scores)
  recall_scores = cross_val_score(model, X, y, scoring = 'recall', cv = cv)
  recall_scores = list(recall_scores)

  print(print_text)
  print('accuracy score: ' + str(mean(accuracy))[0:5] + " +/- " + str(std(accuracy))[0:5])
  print('f1 score: ' + str(mean(f1_scores))[0:5] + " +/- " + str(std(f1_scores))[0:5])
  print('precision: ' + str(mean(precision_scores))[0:5] + " +/- " + str(std(precision_scores))[0:5])
  print('recall: '+ str(mean(recall_scores))[0:5] + " +/- " + str(std(recall_scores))[0:5] + "\n")

In [ ]:
eval_model(model = GaussianNB(), X = X_train, y = y_train, print_text="Naive Bayes")
eval_model(model = SVC(), X = X_train, y = y_train, print_text="Support Vector Machine")
eval_model(model = RandomForestClassifier(), X = X_train, y = y_train, print_text="Random Forest")
eval_model(model = KNeighborsClassifier(), X = X_train, y = y_train, print_text="KNN")

Naive Bayes
accuracy score: 0.965 +/- 0.006
f1 score: 0.876 +/- 0.024
precision: 0.851 +/- 0.036
recall: 0.905 +/- 0.044

Support Vector Machine
accuracy score: 0.972 +/- 0.007
f1 score: 0.896 +/- 0.029
precision: 0.923 +/- 0.039
recall: 0.871 +/- 0.039

Random Forest
accuracy score: 0.972 +/- 0.005
f1 score: 0.889 +/- 0.025
precision: 0.965 +/- 0.021
recall: 0.820 +/- 0.051

KNN
accuracy score: 0.920 +/- 0.009
f1 score: 0.591 +/- 0.057
precision: 0.996 +/- 0.012
recall: 0.423 +/- 0.057



## Stuff that didn't work

In [ ]:
### Remove duplicate / correlated features
def remove_correlated_features(X, corr_threshold=0.9):
  corr = X.corr()
  drop_columns = np.full(corr.shape[0], False, dtype=bool)
  for i in range(corr.shape[0]):
      for j in range(i + 1, corr.shape[0]):
          if corr.iloc[i, j] >= corr_threshold:
              drop_columns[j] = True
  columns_dropped = X.columns[drop_columns]
  print("dropping",columns_dropped)
  X_drop = X.drop(columns_dropped, axis=1)
  return X_drop

In [ ]:
import numpy as np

X = doc_term_mat2.drop(['doc_id', 'category'], axis=1)
X_drop = remove_correlated_features(X, corr_threshold=0.9)

Remove less significant features.

In [ ]:
#regression_ols = sm.OLS(y_train, X_train).fit()
#regression_ols.summary()

In [ ]:
import statsmodels.api as sm
def remove_less_significant_features(X, Y, sl=0.05):
  regression_ols = None
  columns_dropped = np.array([])
  #for itr in range(0, len(X.columns)):
  for itr in range(0, 60):
    if itr % 5 == 0:
      print("Iteration number " + str(itr))
    regression_ols = sm.OLS(Y, X).fit()
    max_col = regression_ols.pvalues.idxmax()
    max_val = regression_ols.pvalues.max()
    if max_val > sl:
        X.drop(max_col, axis='columns', inplace=True)
        columns_dropped = np.append(columns_dropped, [max_col])
    else:
        break
  regression_ols.summary()
  return columns_dropped

In [ ]:
start_time = time.time()
cols_dropped =  remove_less_significant_features(X_train,y_train, sl=0.05)
print("--- %s time elapsed ---" % str(timedelta(seconds=time.time() - start_time)))

In [ ]:
cols_dropped